In [ ]:
# %pip install PyPDF2
# %pip install reportlab
# %pip install tqdm

In [ ]:
from PyPDF2 import PdfReader, PdfWriter, PageObject
from reportlab.pdfgen import canvas
import io
from tqdm import tqdm
import random
import string

In [ ]:
# 定義可能的字符
characters = "123456789"# + string.ascii_lowercase #a~z

#學生人數
student_count = 40

# 生成 student_count 個唯一的亂碼
def generate_unique_codes(count, length):
    codes = set()
    while len(codes) < count:
        code = ''.join(random.choices(characters, k=length))
        codes.add(code)
    return list(codes)

# 呼叫函數
unique_codes = generate_unique_codes(student_count, 5)
print(unique_codes[0])

87517


In [ ]:
#pdf
writer = PdfWriter()

# 原 PDF(考卷)
input_pdf_path = r"D:\Download\temp2.pdf"
reader = PdfReader(input_pdf_path)
first_page = reader.pages[0]

# 獲取頁面尺寸
width = float(first_page.mediabox.width)
height = float(first_page.mediabox.height)
custom_pagesize=(width, height)

# 合併後的 PDF(產生亂碼)
output_pdf_path = r"D:\Download\merged_file.pdf"

for random_code in tqdm(unique_codes,desc="Processing:"):

    # 創建包含亂碼的透明 PDF
    packet = io.BytesIO()
    can = canvas.Canvas(packet, pagesize=custom_pagesize)
    can.setFont("Times-Bold", 14)  # 字體及大小
    x,y=25, 800
    can.drawString(x, y, random_code)

    # 繪製底線（與文字寬度匹配）
    text_width = can.stringWidth(random_code, "Times-Roman", 14)
    line_y = y - 2  # 底線的位置（略低於文字）
    can.line(x, line_y, x + text_width, line_y)

    can.save()
    packet.seek(0)

    # 讀取含亂碼的透明 PDF
    overlay_pdf = PdfReader(packet)
    overlay_page = overlay_pdf.pages[0]

    # 將亂碼覆蓋到第一頁
    for i, page in enumerate(reader.pages):
        if i == 0:  # 僅在第一頁添加亂碼
            # 合併頁面
            page_copy = PageObject.create_blank_page(width=page.mediabox.width, height=page.mediabox.height)
            page_copy.merge_page(page)
            page_copy.merge_page(overlay_page)
            writer.add_page(page_copy)
        else:
            # 直接添加其他頁面
            writer.add_page(page)
     
# 保存修改後的 PDF
with open(output_pdf_path, "wb") as output_pdf:
    writer.write(output_pdf)

print(f"修改完成，文件已儲存至：{output_pdf_path}")


Processing:: 100%|██████████| 40/40 [00:06<00:00,  6.07it/s]


修改完成，文件已儲存至：D:\Download\merged_file.pdf
